# Clasificación de texto utilizando LSTM


In [ ]:
#Importamos nuestras librerias
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
#Descargamos los datos
!curl -O http://srodriguez.me/Datasets/imdb.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 25.7M  100 25.7M    0     0  8485k      0  0:00:03  0:00:03 --:--:-- 8485k


In [ ]:
#Descomprimimos los datos
!unzip imdb.zip

Archive:  imdb.zip
  inflating: IMDB Dataset.csv        


# Carga de datos

En este caso vamos a cargar los datos de IMDB, para clasificación de sentimiento 

In [ ]:
#Generamos nuestro Dataframe, leyendo el archivo .csv
df = pd.read_csv("IMDB Dataset.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
# Mostramos los primero 5 elementos
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
#Muestreamos 10.000 reviews en vez de procesar todo el conjunto de dato
sub_df = df.sample(30000)
sub_df.reset_index(drop=True,inplace=True) #Reiniciamos los indices de este nuevo df

In [ ]:
# Se transforma el texto de las etiquetas en valores numericos para el proceso de aprendizaje
sub_df['sentiment'] = sub_df['sentiment'].map({'positive':1,'negative':0})

In [ ]:
#Vemos el primer review del nuevo set de datos
sub_df.review[0]

'think round applause order whoever pieced together trailer rogue pictures\' latest release, \'the return\'. myself, along everyone else duped believing fact horror film. contrary though, actually supernatural thriller. bad least bit thrilling.<br /><br />\'the return\' stars sarah michelle gellar joanna mills, young woman personal problems since age eleven. age began haunting visions depicting murder woman never met. texas business trip, led visions murdered woman\'s hometown, la salle. comes face face another person frequently appeared visions. man name terry stahl, played peter o\'brien. joanna desperate search answers. search could end result murder.<br /><br />i really know begin folks. mention first? atrocious acting, hideous directing, terribly bland story? matter one choose point behind same: simply suck. adam sussman\'s screenplay downright moronic. interesting. compelling. plain unpleasant. kept waiting something jumpstart "film" (i\'ve placed quotations around film believe \

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Cleaning the texts
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


"""review=[ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
review=' '.join(review)
corpus.append(review)"""

stopwords_set = set(stopwords.words('english'))

sub_df['review'] = sub_df['review'].apply(lambda x: " ".join([word for word in x.lower().split(" ") if not word in stopwords_set]))

In [ ]:
#Importamos la funcion para hacer separacion de los distintos conjuntos de entrenamiento
from sklearn.model_selection import train_test_split

In [ ]:
#Separamos un 30% de datos para test
temp_df, test_df = train_test_split(sub_df,test_size=0.3,random_state=42)
#y de training, sacamos un 10% para validación
train_df, val_df = train_test_split(temp_df,test_size=0.1,random_state=42)

train_df.reset_index(drop=True,inplace=True)
val_df.reset_index(drop=True,inplace=True)
test_df.reset_index(drop=True,inplace=True)

In [ ]:
max_features = 60000  # Considerar las primeras 20000 palabras para generar un diccionario
maxlen = 200  # Considerar las primeras 200 palabras de cada review

#Generamos nuestro tokenizador, el cual nos va a permitir generar nuestro diccionario
tokenizer = keras.preprocessing.text.Tokenizer(num_words=max_features, oov_token='<unk>', )

In [ ]:
#Construimos el vocabulario
tokenizer.fit_on_texts(train_df['review'])

In [ ]:
#Vemos que palabra corresponde al indice numero 2
tokenizer.index_word[3]

'movie'

In [ ]:
#Vemos que indice que corresponde la palabra 'the'
tokenizer.word_index['red']

715

In [ ]:
#Generamos las secuencias al transformar de texto a valores numericos
secuencias = tokenizer.texts_to_sequences(train_df['review'])


In [ ]:
#Fijamos las secuencias en un largo especifico, añadiendo los token de padding '<pad>'
secuencias_padded = keras.preprocessing.sequence.pad_sequences(secuencias,maxlen= maxlen)
secuencias_padded.shape

(18900, 200)

In [ ]:
maxlen

200

In [ ]:
# Transformamos el texto a secuencia para los conjuntos de validacion y testing
val_seq = tokenizer.texts_to_sequences(val_df['review'])
val_seq_padded = keras.preprocessing.sequence.pad_sequences(val_seq,maxlen= maxlen)

test_seq = tokenizer.texts_to_sequences(test_df['review'])
test_seq_padded = keras.preprocessing.sequence.pad_sequences(test_seq,maxlen= maxlen)

In [ ]:
#Generamos nuestras entradas para la red
X_train = secuencias_padded
y_train =train_df['sentiment']

X_val = val_seq_padded
y_val = val_df['sentiment']

X_test = test_seq_padded
y_test = test_df['sentiment']

In [ ]:
#Capa de entrada, la cual recibira los arreglos de interos (indices del vocabulario)
inputs = keras.Input(shape=(maxlen,), dtype="int32")
# Transformamos cada indice, en su vector de palabras correspondiente
x = layers.Embedding(max_features + 1, 128)(inputs)
#Añadimos una capa de LSTM
x = layers.Bidirectional(layers.LSTM(64))(x)

# Añadimos la capa de salida, 1 neurona de salida debido a que es clasificación binaria
# Ademas, utilizamos la funcion de activación sigmoidea para que arroje la probabilidad
outputs = layers.Dense(1, activation="sigmoid")(x)

#Generamos el modelo
model = keras.Model(inputs, outputs)
model.summary() # E imprimimos el modelo 


Model: "functional_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 200, 128)          2560128   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total params: 2,659,073
Trainable params: 2,659,073
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Compilamos el modelo, seleccionamos el optimizado, la funcion de perdida y la metrica de exactitud
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
#Ajustamos el modelo, utilizando los conjuntos de entrenamiento y validamos con el conjunto de validación
#Entrenamos por 5 "Epocas"
model.fit(X_train, y_train, batch_size=16, epochs=2, validation_data=(X_val, y_val))


Epoch 1/2
1182/1182 [==============================] - 52s 44ms/step - loss: 0.4300 - accuracy: 0.8019 - val_loss: 0.3646 - val_accuracy: 0.8367
Epoch 2/2
1182/1182 [==============================] - 52s 44ms/step - loss: 0.2423 - accuracy: 0.9055 - val_loss: 0.3454 - val_accuracy: 0.8633


In [ ]:
#Realizamos la prediccion del modelo y vemos el output (Son probabilidades)
y_pred = model.predict(X_test,batch_size=16,verbose=1)
y_pred

563/563 [==============================] - 4s 8ms/step


array([[0.9856192 ],
       [0.30775046],
       [0.99553776],
       ...,
       [0.02330089],
       [0.14930056],
       [0.9933368 ]], dtype=float32)

In [ ]:
#Transformamos la prediccion en valores binarios al preguntar por todos los valores
# mayores o iguales a 0.5
y_pred = y_pred >= 0.5
y_pred

array([[ True],
       [False],
       [ True],
       ...,
       [False],
       [False],
       [ True]])

In [ ]:
#Transformamos los valores binarios a '0' y '1' utilizando una representación de enteros
y_pred = y_pred.astype(int)
y_pred

array([[1],
       [0],
       [1],
       ...,
       [0],
       [0],
       [1]])

In [ ]:
#Aplanamos el vector para que quede de forma unidimensional
y_pred = y_pred.reshape(-1)
y_pred

array([1, 0, 1, ..., 0, 0, 1])

In [ ]:
#Importamos la libreria de métricas de scikit-learn
from sklearn import metrics

In [ ]:
model.save("modelo.h5") #Salvamos el modelo (o los pesos de este) para un futuro uso

In [ ]:
model.load_weights("modelo.h5") #Con una arqutectura similar, rescatamos el modelo +

In [ ]:
#Imprimimos los valores de prueba, solo por sanity check
print(y_test.values)

[1 0 1 ... 0 0 1]


In [ ]:
#Imprimimos las metricas
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

0.869
              precision    recall  f1-score   support

           0       0.89      0.85      0.87      4598
           1       0.85      0.89      0.87      4402

    accuracy                           0.87      9000
   macro avg       0.87      0.87      0.87      9000
weighted avg       0.87      0.87      0.87      9000



In [ ]:
print(metrics.confusion_matrix(y_test, y_pred))

[[3925  673]
 [ 506 3896]]


# Utilizando vectores de palabras pre-entrenados

https://github.com/RaRe-Technologies/gensim-data <- De aca podemos obtener modelos pre-entrenados soportados por la misma libreria de GenSim

In [ ]:
import gensim.downloader as api
#En este caso, vamos a utilizar los vectores entrenados en wikipedia mediante el algoritmo Globe
#wv = api.load('glove-wiki-gigaword-300')

In [ ]:
#Verificamos que la palabra 'the' exista y mostramos los primeros 10 valores
wv['the'][:10]

array([ 0.04656  ,  0.21318  , -0.0074364, -0.45854  , -0.035639 ,
        0.23643  , -0.28836  ,  0.21521  , -0.13486  , -1.6413   ],
      dtype=float32)

In [ ]:
# Instanciamos una matriz incializada al azar con un vocabulario de max_features + 1
# Esto ultimo es para considerar el vector asociado al token '<pad>'
emb_matrix = np.random.rand(max_features + 1,300) #El largo de nuestro vector

emb_matrix[0][:10]

array([0.61106771, 0.74995914, 0.63748152, 0.44308281, 0.5318564 ,
       0.78282585, 0.66913958, 0.80441194, 0.24493687, 0.56856688])

In [ ]:
#Importamos una libreria para hacer seguimiento del procesos de traspaso de embeddings
from tqdm.notebook import tqdm

In [ ]:
#Iteramos por cada una de las palabras
for i in tqdm(range(1, max_features + 1)):
  #Obtenemos la palabra correspondiente al indice i
  word = tokenizer.index_word[i]
  if word in wv: #Preguntamos si la palabra esta en el modelo de vectores de palabra
    emb_matrix[i] = wv[word] # Asignamos el valor a la fila al vector de palabra


In [ ]:
#Comprobamos que el la matriz de embeddings populada, el indice 2 (para la palabra 'the') 
#corresponde al valor que vimos anteriormente
emb_matrix[2:6,:5]

array([[ 0.40287   , -0.48699   ,  0.091598  , -0.071945  , -0.063545  ],
       [-0.138     , -0.12203   ,  0.0054643 , -0.010215  ,  0.13134   ],
       [-0.030351  , -0.17344999, -0.097576  , -0.20939   , -0.1964    ],
       [-0.36756   ,  0.39500001, -0.27034   , -0.14816999, -0.026378  ]])

In [ ]:
#Capa de entrada, la cual recibira los arreglos de interos (indices del vocabulario)
inputs = keras.Input(shape=(maxlen,), dtype="int32")
# Transformamos cada indice, en su vector de palabras correspondiente
# Pero esta vez, utilizando el parametro weights, inicializamos esta capa de vectores
# Con los pesos extraidos del modelo pre-entrenado
# Aparte, definimos que esta capa no sea entrenable, o sea que los valores de los pesos
# No se vayan ajustando en cada iteración de entrenamiento
x = layers.Embedding(max_features + 1, 300,weights=[emb_matrix], trainable=False)(inputs)
#Añadimos una capa de LSTM
x = layers.LSTM(64)(x)
# Añadimos la capa de salida, 1 neurona de salida debido a que es clasificación binaria
# Ademas, utilizamos la funcion de activación sigmoidea para que arroje la probabilidad
outputs = layers.Dense(1, activation="sigmoid")(x)

#Generamos el modelo
model2 = keras.Model(inputs, outputs)
model2.summary()

Model: "functional_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 200)]             0         
_________________________________________________________________
embedding_12 (Embedding)     (None, 200, 300)          18000300  
_________________________________________________________________
lstm_12 (LSTM)               (None, 64)                93440     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 65        
Total params: 18,093,805
Trainable params: 93,505
Non-trainable params: 18,000,300
_________________________________________________________________


In [ ]:
#Evaluamos y obtenemos las metricas

In [ ]:
model2.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model2.fit(X_train, y_train, batch_size=16, epochs=6, validation_data=(X_val, y_val))


Epoch 1/6
1182/1182 [==============================] - 12s 11ms/step - loss: 0.4460 - accuracy: 0.7923 - val_loss: 0.3489 - val_accuracy: 0.8452
Epoch 2/6
1182/1182 [==============================] - 12s 10ms/step - loss: 0.3290 - accuracy: 0.8649 - val_loss: 0.3161 - val_accuracy: 0.8590
Epoch 3/6
1182/1182 [==============================] - 12s 10ms/step - loss: 0.2725 - accuracy: 0.8898 - val_loss: 0.2969 - val_accuracy: 0.8738
Epoch 4/6
1182/1182 [==============================] - 12s 10ms/step - loss: 0.2281 - accuracy: 0.9053 - val_loss: 0.2943 - val_accuracy: 0.8800
Epoch 5/6
1182/1182 [==============================] - 13s 11ms/step - loss: 0.1805 - accuracy: 0.9311 - val_loss: 0.2885 - val_accuracy: 0.8743
Epoch 6/6
 638/1182 [===============>..............] - ETA: 5s - loss: 0.1358 - accuracy: 0.9493

KeyboardInterrupt: ignored

In [ ]:
y_pred = model2.predict(X_test,batch_size=16,verbose=1)
y_pred = y_pred >= 0.5
y_pred = y_pred.astype(np.int8).reshape(-1)

563/563 [==============================] - 3s 5ms/step


In [ ]:
print(metrics.accuracy_score(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred))

0.8842222222222222
              precision    recall  f1-score   support

           0       0.88      0.90      0.89      4598
           1       0.89      0.87      0.88      4402

    accuracy                           0.88      9000
   macro avg       0.88      0.88      0.88      9000
weighted avg       0.88      0.88      0.88      9000



In [ ]:
# Como modelar texto del usuario para hacer las pruebas


texto_ejemplo = "i love so much this movie" # Recepcion de texto

##
# Preprocesamos (quitamos tildes, sacamos stopwords, dejamos todas las palabras)
##

ej_seq = tokenizer.texts_to_sequences([texto_ejemplo])
ej_seq_padded = keras.preprocessing.sequence.pad_sequences(ej_seq,maxlen= maxlen)

print(ej_seq)
print(ej_seq_padded)

[[36, 41, 174, 17, 43, 3]]
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0  36  41 174  17
   43   3]]


In [ ]:
model2.predict(ej_seq_padded)
